# Excelin ohjaus Pythonilla

Exceliä ohjataan Pythonista käsin **xlwings**-kirjastoa käyttäen. Annetaan kirjastolle vakiintuneen tavan mukaisesti lempinimi **xw**.

In [1]:
import pandas as pd
import xlwings as xw

## Excelin oliomalli

Excel-ohjelmoinnin aluksi täytyy sisäistää Excelin oliomallin keskeisimmät osat. Excelin oliomallissa oliot ovat hierarkisesti järjestyneet (suluissa xlwings-kirjaston käyttämät nimet):

* Hierarkiassa ylimpänä on Applications (**apps**)-kokoelma, joka sisältää yksittäiset avoinna olevat Excel-instanssit.
* Excel-instanssi sisältää Workbooks (**books**)-kokoelman, joka sisältää Excel-instanssin avoimet työkirjat.
* Työkirja sisältää Worksheets (**sheets**)-kokoelman, joka sisältää työkirjan taulukkovälilehdet.
* Taulukkovälilehti sisältää solualueita (**range**).

## Uuden Excel-instanssin luominen

Uuden Excel-instanssin voi luoda xlwingsin App-luokan oliona:

In [2]:
# Luodaan uusi Excel-instanssi
app = xw.App(visible=False)

Jos instanssi on näkyvillä ja käyttäjä menee vahingossa sorkkimaan sitä kesken tietojen kirjoituksen, niin ohjelma voi kaatua virheilmoitukseen. Excel-instanssi kannattaa tämän vuoksi luoda näkymättömänä (visible=False) ja tuoda se näkyviin vasta sen jälkeen, kun kaikki tiedot on kirjoitettu. Kokeilu- ja testausvaiheessa Excel-instanssi kannattaa kuitenkin pitää näkyvänä (visible=True), jotta komentojen vaikutuksia pystyy seuraamaan.

## Työkirjat

Uuden Excel-instanssin myötä syntyy työkirja. Seuraava komento nimeää työkirjan wb1-nimiseksi. Tämä ei tarkoita tallennusnimeä, vaan nimeä, jolla koodissa työkirjaan pystyy viittaamaan:

In [3]:
# Excel-instanssin myötä syntyi myös työkirja, jolle annetaan tässä nimi wb1
wb1 = app.books[0]

Täysin uuden työkirjan saa aikaiseksi luomalla uuden Book-luokan olion:

In [4]:
# Luodaan toinen työkirja
wb2 = xw.Book()

## Taulukkovälilehdet

Työkirjan taulukkovälilehtiin pystyy viittaamaan sheets-kokoelman kautta. Tässä käytetään Pythonista tuttua indeksointia, joka alkaa nollasta:

In [5]:
# Näillä nimillä voidaan viitata työkirjojen taulukkovälilehtiin
wb1_ws1 = wb1.sheets[0]
wb2_ws1 = wb2.sheets[0]

Edellä olevan perusteella koodissa voi myöhemmin viitata taulukkovälilehtiin nimillä wb1_ws1 ja wb2_ws1. Nämä eivät kuitenkaan ole Excelissä taulukkovälilehtien nimiä, vaan nimiä, joilla koodissa viitataan taulukkovälilehtiin.

Täysin uuden taulukkovälilehden voi luoda **sheets**-kokoelman **add**-funktiolla:

In [6]:
# Lisätään uusi taulukkovälilehti
wb2_ws2 = wb2.sheets.add(name='toinen', after=wb2.sheets.count)

wb2_ws2 on nimi, jolla välilehteen voi Python-koodissa viitata ja name-parametrin arvo ’toinen’ on taulukkovälilehden nimi Excelissä. Yllä taulukkovälilehti lisätään after-parametrin arvon ansiosta oikealle viimeiseksi taulukkovälilehdeksi.

## Solut

Tiedon kirjoittaminen soluun onnistuu **range**-kokoelman **value**-ominaisuuden avulla:

In [7]:
# Kirjoitetaan soluihin
wb1_ws1.range('B3').value = 'x'
wb1_ws1.range('C3').value = 'y'

Excel-tyylinen soluviittaus (esim. ’C3’) ei kaikissa tapauksissa sovi tilanteeseen. Esimerkiksi seuraavassa for-silmukassa täytyy käyttää toisenlaista viittaustapaa (rivin ja sarakkeen numero sulkujen sisällä):

In [8]:
# Kirjoitetaan soluihin
for i in range(10):
    wb1_ws1.range((i+4, 2)).value = i
    wb1_ws1.range((i+4, 3)).value = f'=exp(B{i+4})'

Katso esimerkkejä erilaisista viittaustavoista solualueeseen: https://docs.xlwings.org/en/latest/api/range.html

Myös solualueen lukeminen onnistuu value-ominaisuuden avulla (**expand**-funktio laajentaa valinnan solua B3 ympäröiviin soluihin, jotka eivät ole tyhjiä):

In [9]:
# Luetaan data Excelistä Pythoniin
data = wb1_ws1.range('B3').expand().value
data

[['x', 'y'],
 [0.0, 1.0],
 [1.0, 2.718281828459045],
 [2.0, 7.38905609893065],
 [3.0, 20.085536923187668],
 [4.0, 54.598150033144236],
 [5.0, 148.4131591025766],
 [6.0, 403.4287934927351],
 [7.0, 1096.6331584284585],
 [8.0, 2980.9579870417283],
 [9.0, 8103.083927575384]]

Jos halutaan lukea solualue tietokehykseksi, voidaan käyttää **options**-lisäasetuksia:

In [10]:
# Luetaan data pandas-dataframeen
# Oletusarvona vasemmanpuoleisesta sarakkeesta tulee indeksi
df = wb1_ws1.range('B3').expand().options(pd.DataFrame, index=False).value
df

,x,y
0,0.0,1.000000
1,1.0,2.718282
2,2.0,7.389056
3,3.0,20.085537
4,4.0,54.598150
5,5.0,148.413159
6,6.0,403.428793
7,7.0,1096.633158
8,8.0,2980.957987
9,9.0,8103.083928


Lisätietoa lisäasetuksista https://docs.xlwings.org/en/latest/converters.html

## Tallennus ja Excel-instannin sulkeminen

Työkirjan voin tallentaa **save**-funktiolla:

In [11]:
# Työkirjan tallennus
wb1.save('pyex/output/excel1.xlsx')

Sen jälkeen, kun kaikki tiedot on kirjoitettu niin Excel-instanssin voi tuoda näkyville:

In [12]:
app.visible = True

Jos Excel-instanssia ei ole tarvetta jättää auki, niin sen voi sulkea komennolla

In [13]:
# Excel-instanssin sulkeminen
app.quit()

Jos haluat sulkea kaikki avoimet Excelit ilman tallennusta, niin voit käyttää for-silmukkaa:

In [14]:
# Jos haluat sulkea ilman tallennusta kaikki avoinna olevat Excelit (myös piilotetut), niin voit käyttää for-silmukkaa:
# Näin pääsee kerralla eroon kaikista Excel-instansseista.

#for app in xw.apps:
#    app.quit()

Lisätietoa käytetyistä ja muista mahdollisista toiminnoista löydät xlwingsin Python **API**sta (API=ohjelmointirajapinta): https://docs.xlwings.org/en/latest/api/index.html.

## load ja view

Xlwingsin yllättävin ominaisuus on **load**- ja **view**-funktioiden mukanaan tuoma interaktiivisuus:

Jos Excelin aktiivisella taulukkovälilehdellä on data, jonka sisältä on valittu yksi solu, niin **xw.load()** lataa aineiston pandaksen tietokehykseen. Jos valittuna on useampia soluja, niin ladatuksi tulee valitun solualueen sisältämä aineisto. Tutustutaan tähän tarkemmin seuraavassa.

### Datan lataaminen Excelistä

Jos data on auki Excelin aktiivisella taulukkovälilehdellä ja datan sisältä on täsmälleen yksi solu valittuna, niin datan lataaminen dataframeen on helppoa.

**Tärkeää!** Tämä toimii vain jos Excelissä on sopiva data aktiivisena!

Ennen seuraavien koodisolujen suorittamista oletetaan, että käyttäjä on avannut materiaalipaketin data/aineisto.xlsx-tiedoston.

In [16]:
## Seuraavissa oletetaan, että käyttäjällä on Excel auki ja sopiva tiedosto avattuna sekä aineistosta yksi solu valittuna.

df = xw.load()
df

,Myymälän sijainti on hyvä,Aukioloajat ovat sopivat,Myymälä on hyvin johdettu,Helppo tavoittaa puhelimella,Hyödylliset nettisivut,Työntekijät ovat ystävällisiä,Työntekijät ovat avuliaita,Työntekijät ovat asiantuntevia,Hinnat ovat kilpailukykyiset,Hyvä tuotevalikoima,Pidän TV-mainoksistanne,Tuotteiden laatu on hyvä,Yleisesti ottaen tyytyväinen,Suosittelisin tuttavilleni
Numero,,,,,,,,,,,,,,
1.0,2.0,5.0,1.0,1.0,4.0,2.0,4.0,3.0,3.0,2.0,2.0,5.0,2.0,3.0
2.0,2.0,2.0,5.0,5.0,4.0,2.0,1.0,5.0,5.0,2.0,3.0,4.0,2.0,5.0
3.0,2.0,4.0,3.0,3.0,1.0,1.0,4.0,4.0,4.0,2.0,2.0,2.0,2.0,4.0
4.0,2.0,4.0,5.0,4.0,5.0,3.0,2.0,5.0,4.0,4.0,1.0,5.0,4.0,4.0
5.0,3.0,4.0,4.0,4.0,5.0,1.0,4.0,1.0,4.0,1.0,2.0,1.0,1.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67.0,1.0,3.0,5.0,1.0,2.0,2.0,4.0,1.0,3.0,4.0,5.0,5.0,4.0,3.0
68.0,3.0,4.0,3.0,5.0,5.0,4.0,4.0,3.0,2.0,4.0,2.0,2.0,4.0,2.0
69.0,1.0,3.0,5.0,4.0,5.0,5.0,5.0,1.0,1.0,5.0,1.0,2.0,5.0,1.0


### Tulokset Exceliin

**xw.view()** puolestaan vie tulokset uuteen Excel-työkirjaan.

In [17]:
# T vaihtaa taulukon rivit ja sarakkeet päittäin
tunnusluvut = df.describe().T

# Suomenkieliset nimet tilastollisille tunnusluvuille
tunnusluvut.columns = ['Lukumäärä', 'Keskiarvo', 'Keskihajonta', 'Pienin', 
                     'Alaneljännes', 'Mediaani', 'Yläneljännes', 'Suurin']

xw.view(tunnusluvut, table=False)

### Tulosten muotoilu

Xlwings APIn (API=Application Programming Interface) kautta voi ohjelmoida Exceliä. Opettele lukemaan APIn dokumentaatiota: https://docs.xlwings.org/en/latest/api/index.html

Kaikkia Excelin toimintoja (mm. solujen reunaviivat ja solun sisällön tasaus) ei ole vielä toteutettu Xlwings APIin. Puuttuviin toimintoihin pääsee käsiksi Windowsissa pywin32:n kautta ja Macissa appscriptin kautta (näiden osalta siis erilainen koodi Windowsiin ja Maciin).

Värikoodit voit antaa RGB-muodossa (red, green, blue). Värikoodeja löydät esimerkiksi sivulta https://www.rapidtables.com/web/color/RGB_Color.html

Huomaa seuraavassa myös, että nimettyihin Excelin taulukkovälilehtiin voi viitata. Lukumuotoiluina käytetään Excelin lukumuotoiluja.

In [18]:
# tunnusluvut-taulukon korkeus ja leveys soluina
korkeus = tunnusluvut.shape[0]+1
leveys = tunnusluvut.shape[1]+1

# Nimetään Excelin taulukkovälilehti
xw.sheets[0].name = 'Tunnusluvut'

# Asetetaan lukumuotoilu
xw.sheets('Tunnusluvut').range((2, 3), (korkeus, leveys)).number_format = '0,00'

# Muotoillaan otsikkorivi
rng = xw.sheets('Tunnusluvut').range((1, 1), (1, leveys))
rng.font.bold = True
rng.color = (204, 229, 251)

# Sovitetaan sarakkeiden leveydet tilanteeseen sopiviksi
xw.sheets('Tunnusluvut').used_range.autofit()

# Tasaus oikeaan reunaan, toimiin vain Windowsissa
rng.api.HorizontalAlignment = -4152

# Reunaaviiva alapuolelle, toimii vain Windowsissa
rng.api.Borders(9).Weight = 2

MacOS reunaviivat katso: https://github.com/xlwings/xlwings/issues/440

MacOS tasaus katso: https://github.com/xlwings/xlwings/issues/1937

Windowsia varten löydät muotoiluun liittyviä vakioita osoitteesta https://github.com/xlwings/xlwings/blob/main/xlwings/constants.py. Tässä muutama poiminta kyseisestä osoitteesta:

class BorderWeight:

    xlHairline = 1  # from enum XlBorderWeight
    xlMedium = -4138  # from enum XlBorderWeight
    xlThick = 4  # from enum XlBorderWeight
    xlThin = 2  # from enum XlBorderWeight

class BordersIndex:

    xlDiagonalDown = 5  # from enum XlBordersIndex
    xlDiagonalUp = 6  # from enum XlBordersIndex
    xlEdgeBottom = 9  # from enum XlBordersIndex
    xlEdgeLeft = 7  # from enum XlBordersIndex
    xlEdgeRight = 10  # from enum XlBordersIndex
    xlEdgeTop = 8  # from enum XlBordersIndex
    xlInsideHorizontal = 12  # from enum XlBordersIndex
    xlInsideVertical = 11  # from enum XlBordersIndex
    
class HAlign:

    xlHAlignCenter = -4108  # from enum XlHAlign
    xlHAlignCenterAcrossSelection = 7  # from enum XlHAlign
    xlHAlignDistributed = -4117  # from enum XlHAlign
    xlHAlignFill = 5  # from enum XlHAlign
    xlHAlignGeneral = 1  # from enum XlHAlign
    xlHAlignJustify = -4130  # from enum XlHAlign
    xlHAlignLeft = -4131  # from enum XlHAlign
    xlHAlignRight = -4152  # from enum XlHAlign

Tutustu materiaalipaketin muistioon **xlwings2.ipynb**.

## xlsxwriter vai xlwings?

Huomaa ero xlsxwriterin ja xlwingsin välillä:

* xlsxwriter kirjoittaa Excel-tiedostoon (katso materiaalipaketin **pika2.ipynb**)
* xlwings kirjoittaa avoinna olevaan Exceliin (katso materiaalipaketin **pika1.ipynb**)

Näitä kahta ei pidä sekoittaa toisiinsa. Kirjoittamis- ja muotoilukomennot ovat kummassakin omanlaisensa.

Vertaile tiedostojen käsittelyyn liittyviä ominaisuuksia muistioissa pika1.ipynb ja pika2.ipynb. Tämän aihekokonaisuuden soveltamista voi testailla erilaisilla aineistoilla ja muistiolla pika1.ipynb.

Tehtävissä harjoitellaan Excelin ohjausta Pythonilla.

In [19]:
import datetime
print(f'Viimeksi muokattu {datetime.datetime.now():%Y-%m-%d %H:%M}, Juha Nurmonen')

Viimeksi muokattu 2024-04-02 19:20, Juha Nurmonen
